In [ ]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stopwords.words('english'):
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()


from datetime import datetime

def convert_dt(s):
    return datetime.strptime(s, '%m/%d/%Y')

def get_date(d):
    return int(d.days)

In [ ]:
df_train = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/train.csv')
df_train['tag'] = 'train'
print df_train.shape

df_test = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/test.csv') 
df_test['tag'] = 'test'
df_test['Complaint-Status'] ='ign'
print df_test.shape

In [ ]:
df = pd.concat([df_train , df_test],axis= 0)
print df.shape


In [ ]:
df['Date-received'] = df['Date-received'].apply(convert_dt)
df['Date-sent-to-company'] = df['Date-sent-to-company'].apply(convert_dt)

In [ ]:
df['date_sub'] = (df['Date-sent-to-company'] -  df['Date-received'])

In [ ]:
df['date_sub'] = df['date_sub'].apply(get_date) 

In [ ]:
df.head()

In [ ]:
# df['Complaint-reason'] = df['Complaint-reason'].apply(preprocess_data)

In [ ]:
df_cat = df[['Transaction-Type']]
df_one = pd.get_dummies(df_cat)

In [ ]:
df_one.shape

In [ ]:
df_one['Complaint-ID'] = df['Complaint-ID']
df_one['tag'] = df['tag']
df_one['Complaint-Status'] = df['Complaint-Status']
df_one['date_sub'] = df['date_sub']


In [ ]:
complete_df_one_hot = df_one

train_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'train']
train_data.drop(['tag'],inplace=True,axis=1)
test_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'test']
test_data.drop(['tag'],inplace=True,axis =1)

In [ ]:
y_train = train_data['Complaint-Status']
x_train = train_data.drop(['Complaint-Status','Complaint-ID'],axis = 1)

In [ ]:
x_train.shape

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)

principalComponents = pca.fit_transform(x_train)

df_pca = pd.DataFrame(data = principalComponents)

In [ ]:
X  = np.array(df_pca)
y = np.array(y_train)

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.naive_bayes import MultinomialNB,GaussianNB
# from sklearn.linear_model import LogisticRegression
# X_train, X_test, y_train_internal, y_test_internal = train_test_split(df_pca,y_train)

In [ ]:
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
k_fold = StratifiedKFold(n_splits= 50)
skf = StratifiedKFold(n_splits=50)
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)

for train_index, test_index in skf.split(df_pca,y_train):
    X_train, X_test = X[train_index], X[test_index]
    y_train_, y_test = y[train_index], y[test_index]
    
    from sklearn.neighbors import KNeighborsClassifier
    rf = KNeighborsClassifier(n_neighbors=3).fit(X_train,y_train_)

    pred_internal = rf.predict(X_test)
    import sklearn
    ##print pred_intenal
    print sklearn.metrics.f1_score(pred_internal,y_test,average='weighted')

#from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 1000 decision trees
#

#print cross_val_score(rf, df_pca, y_train, cv=k_fold, n_jobs=1)

In [ ]:
len(df_pca)

In [ ]:
rf = RandomForestClassifier().fit(df_pca, y_train)
pred_internal = rf.predict(df_pca)
import sklearn
##print pred_intenal
sklearn.metrics.f1_score(pred_internal,y_train,average='weighted')

In [ ]:
id_ = list(test_data['Complaint-ID'])

In [ ]:
print train_data.shape
test_data.shape
test_data.drop(['Complaint-Status','Complaint-ID'],inplace=True,axis=1)

In [ ]:
test_data.head()
from sklearn.decomposition import PCA

pca = PCA(n_components=3)

principalComponents = pca.fit_transform(test_data)

df_pca = pd.DataFrame(data = principalComponents)
y_pred = rf.predict(df_pca)

In [ ]:
result = pd.DataFrame(id_,columns=['Complaint-ID'])
result['Complaint-Status'] = y_pred

In [ ]:
result.to_csv('/home/prashant/Downloads/c3cc8568-0-dataset/date_fet_tt_fet_pca_1a.csv',index=None)

In [ ]:
# TESTING DATA ANALYSIS
% matplotlib inline
df_test['Transaction-Type'].value_counts()


In [ ]:
df_train['Transaction-Type'].value_counts()

In [ ]:
df_test['Consumer-complaint-summary']